In [ ]:
%load_ext sql
%sql sqlite://///home/mikew/extracover/male_t20.db

In [ ]:
%config SqlMagic.displaylimit=20

## Run distribution by phase of innings

In [ ]:
qry = """
WITH phased AS (
    SELECT
        *
    ,   CASE
        WHEN over BETWEEN 0 AND 5 THEN '0 powerplay'
        WHEN over BETWEEN 6 AND 17 THEN '1 mid-innings'
        ELSE '2 last two'
        END as phase
    FROM balls
)
SELECT phase, batter_runs, COUNT(*) AS count
FROM phased
WHERE innings < 2
AND extra_type = ''
AND batter_runs in (0,1,2,3,4,6)
GROUP BY phase, batter_runs
"""
result = %sql {{qry}}
df=result.DataFrame()
df.pivot(columns="phase", index="batter_runs", values="count")

## Average runs by over

In [ ]:
sql= """
WITH over_runs AS (
SELECT
	over
,	CAST(SUM(batter_runs) + SUM(extra_runs) AS FLOAT) AS runs
FROM balls
WHERE innings = 0
GROUP BY match_id, over
HAVING MAX(ball) = 5
)
SELECT
	over
,	SUM(runs) / COUNT(*) AS avg_runs
FROM over_runs
GROUP BY over
ORDER BY over
"""

result=%sql {{sql}}
df=result.DataFrame()
df.plot(x="over", y="avg_runs", kind="bar")

## Batsman "Aggression"

First, what's the average *batter only* runs per ball in each over?

In [ ]:
sql= """
WITH over_runs AS (
SELECT
	over
,	CAST(SUM(batter_runs) AS FLOAT) AS runs
FROM balls
WHERE innings = 0
GROUP BY match_id, over
HAVING MAX(ball) = 5
)
SELECT
	over
,	SUM(runs) / COUNT(*) / 6 AS avg_runs
FROM over_runs
GROUP BY over
ORDER BY over
"""

result=%sql {{sql}}
df=result.DataFrame()
df.plot(x="over", y="avg_runs", kind="bar")

Now, take a batsman - #137 CH Gayle has the most ball faced (and is, or was, notoriously aggressive)

In [ ]:
sql = """SELECT
	over
,	COUNT(*)
,	SUM(CAST(batter_runs AS FLOAT)) / COUNT(*) AS avg_runs
FROM balls
WHERE batter = 137
GROUP BY over
ORDER BY over"""

result=%sql {{sql}}
df_gayle=result.DataFrame()
gayle = df_gayle["avg_runs"] - df["avg_runs"]
gayle.plot(kind="bar")
gayle.sum()

...or Virat Kohli (coincidentally #138 at time of computing)

In [ ]:
sql = """
SELECT
	over
,	COUNT(*)
,	SUM(CAST(batter_runs AS FLOAT)) / COUNT(*) AS avg_runs
FROM balls
WHERE batter = 138
GROUP BY over
ORDER BY over"""

result=%sql {{sql}}
df_kohli=result.DataFrame()
kohli = df_kohli["avg_runs"] - df["avg_runs"]
kohli.plot(kind="bar")
kohli.sum()

Can we (usefully) fir some kind of curve (straight line) to that?

In [ ]:
df_kohli

In [ ]:
df_gayle